In [17]:
# ライブラリのインポート
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os

In [21]:
# ナップザック問題を解く関数（アトラクション1個目は移動時間を加えない）
def knapsack(max_time, required_times, popularity, waiting_times, move_time):
    n = len(required_times)
    dp = np.zeros((n+1, max_time+1))
    keep = np.zeros((n+1, max_time+1), dtype=bool)

    for i in range(1, n+1):
        for w in range(max_time+1):
            # アトラクション1個目は移動時間を加えない
            total_time = required_times[i-1] + waiting_times[i-1] + (move_time if i > 1 else 0)
            if total_time <= w:
                if dp[i-1][w] < dp[i-1][w-total_time] + popularity[i-1]:
                    dp[i][w] = dp[i-1][w-total_time] + popularity[i-1]
                    keep[i][w] = True
                else:
                    dp[i][w] = dp[i-1][w]
            else:
                dp[i][w] = dp[i-1][w]

    # 解を復元する
    res = []
    times = []
    w = max_time
    for i in range(n, 0, -1):
        if keep[i][w]:
            res.append(attractions[i-1])
            total_time = required_times[i-1] + waiting_times[i-1] + (move_time if i > 1 else 0)
            w -= total_time
            times.append(w)

    res.reverse()
    times.reverse()
    return res, times, dp[n][max_time]

In [26]:
# アトラクションの所要時間と人気度
attractions = ['ソアリン', 'トイストーリーマニア', 'タワーオブテラー', 'センターオブジアース', 'インディージョーンズ', 'レイジングスピリッツ', 'マジックランプシアター', 'タートルトーク']
required_times = [5, 10, 5, 5, 5, 5, 25, 30]
popularity = [84, 69, 55, 45, 33, 28, 19, 22]
max_time = 705  # パーク内にいる時間 9:00~20:45 計705分
move_time = 30

In [27]:
# メイン処理
# date_dataフォルダからCSVファイルを取り出して処理
folder_path = 'file'
file_names = sorted(os.listdir(folder_path))  # フォルダ名をソート
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    if file_path.endswith('.csv'):
        data = pd.read_csv(file_path)
        start_time = datetime.strptime(data.iloc[0]['時間'], '%Y-%m-%d %H:%M:%S')

        waiting_times = [row[attr] for attr in attractions for index, row in data.iterrows()]
        best_plan, best_times, max_popularity = knapsack(max_time, required_times, popularity, waiting_times, move_time=30)

        ride_times = [start_time + timedelta(minutes=(max_time - t)) for t in best_times]
        ride_schedule = list(zip(ride_times, best_plan))
        ride_schedule.sort(key=lambda x: x[0])

        print(f"フォルダ: {file_name}")
        for time, attraction in ride_schedule:
            print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} に {attraction} に乗ります")
        print("最大人気度スコア:", max_popularity)
        print("-" * 40)


フォルダ: 2022-07-01.csv
2022-07-01 11:35:00 に インディージョーンズ に乗ります
2022-07-01 14:00:00 に センターオブジアース に乗ります
2022-07-01 16:25:00 に タワーオブテラー に乗ります
2022-07-01 18:25:00 に トイストーリーマニア に乗ります
2022-07-01 20:05:00 に ソアリン に乗ります
最大人気度スコア: 286.0
----------------------------------------
